In [1]:
import pandas as pd
import requests
import json
import os

In [2]:
#UTSW proxies
proxies = {"http": "http://proxy.swmed.edu:3128",
           "https": "https://proxy.swmed.edu:3128"}

In [3]:
df = pd.DataFrame(data=os.listdir('NCTFiles'))
df.columns = ['NCTCode']
df['NCTCode'] = df.NCTCode.str.replace('.xml', '')
df.head()

,NCTCode
0,NCT02568267
1,NCT02677116
2,NCT02524119
3,NCT02883049
4,NCT02637687


In [4]:
base_link='https://ckb.jax.org/clinicalTrial/show?nctId='
results_df = pd.DataFrame(columns = ['NCTID', 'Tumors', 'Indications'])

for idx in df.index:
    try:
        #Get html
        trial_response = requests.get(base_link + df.loc[idx, 'NCTCode'], proxies=proxies)
        trial_content = trial_response.content
        response_df = pd.read_html(trial_content, index_col=0, flavor='html5lib')

        #Get tumor information
        tumor_string = response_df[0][1]['Indications']
        if '  ' in tumor_string:
            tumors = response_df[0][1]['Indications'].split('  ')
        else:
            tumors = tumor_string

        #Get molecular information
        molecular_indications = response_df[1]['Requirement Type'].reset_index()
        positive = []
        
        if len(molecular_indications.index)>1:
            for idx2 in molecular_indications.index:
                Indication  = molecular_indications.loc[idx2]
                if Indication['Requirement Type'] in ['partial', 'required']:
                    positive.append(Indication['Molecular Profile'])

        elif len(molecular_indications.index)==1:
            if molecular_indications.loc[0]['Requirement Type'] in ['partial', 'required']:
                positive.append(molecular_indications.loc[0]['Molecular Profile'])
                
        else:
            pass
        
        #Return DataFrame
        results_df.loc[idx] = [df.loc[idx, 'NCTCode'], tumors, positive]

    except Exception as e:
        print(e)
        pass

argument of type 'numpy.float64' is not iterable
argument of type 'numpy.float64' is not iterable
argument of type 'numpy.float64' is not iterable
argument of type 'numpy.float64' is not iterable
argument of type 'numpy.float64' is not iterable
argument of type 'numpy.float64' is not iterable
argument of type 'numpy.float64' is not iterable
argument of type 'numpy.float64' is not iterable
argument of type 'numpy.float64' is not iterable
argument of type 'numpy.float64' is not iterable
argument of type 'numpy.float64' is not iterable
argument of type 'numpy.float64' is not iterable
argument of type 'numpy.float64' is not iterable
argument of type 'numpy.float64' is not iterable
argument of type 'numpy.float64' is not iterable
argument of type 'numpy.float64' is not iterable
argument of type 'numpy.float64' is not iterable
argument of type 'numpy.float64' is not iterable
argument of type 'numpy.float64' is not iterable
argument of type 'numpy.float64' is not iterable
argument of type 'nu

In [5]:
results_df.head()

,NCTID,Tumors,Indications
0,NCT02568267,"[non-small cell lung carcinoma, Advanced Solid...","[ALK rearrange, ROS1 rearrange]"
1,NCT02677116,"[Advanced Solid Tumor, central nervous system ...",[]
2,NCT02524119,hepatocellular carcinoma,[RB1 positive]
3,NCT02883049,precursor B lymphoblastic lymphoma/leukemia,[]
4,NCT02637687,"[congenital fibrosarcoma, Advanced Solid Tumor...",[]


In [6]:
results_df.to_csv('Jax_curations.csv')